This rmd file contains the work that was performed in RStudio Desktop to review, clean and analyze data for the Google Analytics Certificate Capstone Project.  This project uses the Cyclistic Data files that are available from the Divvy Data https://www.divvybikes.com/system-data. A more detailed explanation of the project including the steps taken and data visualizations can be found on this site, [Google Data Analytics Certificate Capstone Project](https://vibrantoutlook.wordpress.com/2021/09/19/google-data-analytics-certificate-capstone-project/).

## Install R packages


In [ ]:
install.packages('tidyverse', repos = "http://cran.us.r-project.org")
install.packages("janitor", repos = "http://cran.us.r-project.org")
install.packages("lubridate", repos = "http://cran.us.r-project.org")

### Load packages


In [ ]:
library(tidyverse)
library(janitor)
library(lubridate)

## Upload Data Files
Upload CSV files of monthly bike share data


In [ ]:
August_2020 <- read_csv("../input/cyclistic-bike-share/202008-divvy-tripdata.csv")
September_2020 <- read_csv("../input/cyclistic-bike-share/202009-divvy-tripdata.csv")
October_2020 <- read_csv("../input/cyclistic-bike-share/202010-divvy-tripdata.csv")
November_2020 <- read_csv("../input/cyclistic-bike-share/202011-divvy-tripdata.csv")
December_2020 <- read_csv("../input/cyclistic-bike-share/202012-divvy-tripdata.csv")
January_2021 <- read_csv("../input/cyclistic-bike-share/202101-divvy-tripdata.csv")
February_2021 <- read_csv("../input/cyclistic-bike-share/202102-divvy-tripdata.csv")
March_2021 <- read_csv("../input/cyclistic-bike-share/202103-divvy-tripdata.csv")
April_2021 <- read_csv("../input/cyclistic-bike-share/202104-divvy-tripdata.csv")
May_2021 <- read_csv("../input/cyclistic-2021/202105-divvy-tripdata.csv")
June_2021 <- read_csv("../input/cyclistic-2021/202106-divvy-tripdata.csv")
July_2021 <- read_csv("../input/cyclistic-2021/202107-divvy-tripdata.csv")

Calculate the total number of records in all twelve monthly files


In [ ]:
sum(nrow(July_2021) + nrow(June_2021) + nrow(May_2021) + nrow(April_2021) + nrow(March_2021) + nrow(February_2021) + nrow(January_2021) + nrow(December_2020) + nrow(November_2020) + nrow(October_2020) + nrow(September_2020) + nrow(August_2020))


## Aggregate and Review Data

Aggregate monthly data frames into one data frame


In [ ]:
alltrips <- rbind(August_2020, September_2020, October_2020, November_2020, December_2020, January_2021, February_2021, March_2021, April_2021, May_2021, June_2021, July_2021)

Check to see that the number of rows in the alltrips data frame is the same as in the twelve input files.

In [ ]:
nrow(alltrips)


Review the column names in alltrips data frame


In [ ]:
colnames(alltrips)


Check number of rows and columns in alltrips data frame


In [ ]:
dim(alltrips)


Check the top and bottom of the data frame

In [ ]:
head(alltrips)
tail(alltrips)

Generate a list of columns and data types in the data frame.


In [ ]:
str(alltrips)


Find unique values in member_casual and rideable_type columns and check for irregularities in the naming


In [ ]:
unique(alltrips[c("member_casual")])
unique(alltrips[c("rideable_type")])

Do some investigation to see what the differences are between the three rideable types of bikes.  

Count number of rides by ride type

In [ ]:
table(alltrips$rideable_type)


Create a temporary dataframe and review the top and bottom of the frame for various combinations of rider type and rideable type to see when these trips occurred.  This chunk of code needs to be run 6 times to get all the possible combinations of member_casual and rideable_type.


In [ ]:
temp <- alltrips %>% filter(rideable_type =="docked_bike" & member_casual == "member") %>% arrange(started_at)

In [ ]:
head(temp)
tail(temp)

From the started_at field, create additional columns for Date, Month, Day, Year, Day of the Week.  This allows for more granular analysis of the data by date/day/month.


In [ ]:
alltrips$date <- as.Date(alltrips$started_at)
alltrips$month <- format(as.Date(alltrips$date), "%m")
alltrips$day <- format(as.Date(alltrips$date), "%d")
alltrips$year <- format(as.Date(alltrips$date), "%Y")
alltrips$day_of_week <- format(as.Date(alltrips$date), "%A")

Create a column for duration of rides calculated from start and end time of rides.


In [ ]:
alltrips$ride_length <- difftime(alltrips$ended_at,alltrips$started_at)


Convert ride_length from factor to numeric so that it can be used in calculations.


In [ ]:
is.factor(alltrips$ride_length)
alltrips$ride_length <- as.numeric(as.character(alltrips$ride_length))
is.numeric(alltrips$ride_length)

Check for Duplicate records based on ride_id by counting unique ride_ids and comparing to the number of rows in the data frame.


In [ ]:
alltrips %>% 
distinct(ride_id) %>% 
count()

This number is less than the total number of rides which indicates that they are duplicate ride ids that need to be cleaned.

Count the number of Start Stations by Name.

In [ ]:
alltrips %>% 
distinct(start_station_name) %>% 
count()

Count the number of end stations


In [ ]:
alltrips %>% 
distinct(end_station_name) %>% 
count()

Review Station Names (create a data frame that can be sorted and reviewed)


In [ ]:
start_station_names <- unique(alltrips[c("start_station_name")])
end_station_names <- unique(alltrips[c("end_station_name")])


Remove trailing spaces in start and end station names if they exist.


In [ ]:
start_station_names %>%
  mutate_if(is.character, str_trim)

Trim end station names


In [ ]:
end_station_names %>%
  mutate_if(is.character, str_trim)

After trimming the number of stations remained the same indicating no leading or trailing blanks in the station names.

Count the number of starting latitudes 

In [ ]:
alltrips %>% 
distinct(start_lat) %>% 
count()

The number of unique starting latitudes is much larger than the number of starting stations which means there is not a one-to-one relationship between the lat/long and station names.

## Clean Data

Create a new data frame for the cleaned data, leaving the initial data frame intact.  Remove duplicate records based on ride_id.  

In [ ]:
alltrips_v2 <- distinct(alltrips, ride_id, .keep_all=TRUE)


Remove "bad" data  (Data where trip is less than 60 seconds or more than 864000 seconds (Divvy considers these bikes stolen)).


In [ ]:
alltrips_v2 <- alltrips_v2[!(alltrips_v2$ride_length<60 | alltrips_v2$ride_length>86400),]

Check for missing values in data


In [ ]:
colSums(is.na(alltrips_v2))


Remove rows where the start station, end station, end_lat, or end_lng have missing data.  Put data into a new data frame alltrips_v3


In [ ]:
alltrips_v3 <- alltrips_v2[!(is.na(alltrips_v2$start_station_id) | is.na(alltrips_v2$end_station_id) | is.na(alltrips_v2$ride_id) | is.na(alltrips_v2$rideable_type) | is.na(alltrips_v2$started_at) | is.na(alltrips_v2$ended_at) | is.na(alltrips_v2$end_lat) | is.na(alltrips_v2$end_lng)),]

Remove rows which have DIVVY CASSETTE REPAIR MOBILE STATION or HUBBARD ST BIKE CHECKING or WATSON TESTING DIVVY as start or end station names as they are administrative stations.

In [ ]:
alltrips_v3<- alltrips_v3[!(alltrips_v3$start_station_name == "DIVVY CASSETTE REPAIR MOBILE STATION" | alltrips_v3$start_station_name == "HUBBARD ST BIKE CHECKING (LBS-WH-TEST)" | alltrips_v3$start_station_name == "WATSON TESTING DIVVY" | alltrips_v3$end_station_name == "DIVVY CASSETTE REPAIR MOBILE STATION" | alltrips_v3$end_station_name == "HUBBARD ST BIKE CHECKING (LBS-WH-TEST)" | alltrips_v3$end_station_name == "WATSON TESTING DIVVY"),]

# Analyze Data

Calculate the mean, median, maximum and minimum trip lengths for all users

In [ ]:
mean(alltrips_v3$ride_length)
median(alltrips_v3$ride_length)
max(alltrips_v3$ride_length)
min(alltrips_v3$ride_length)

Show differences between members and casual riders in terms of length of ride (mean, median, maximum and minimum)


In [ ]:
aggregate(alltrips_v3$ride_length ~ alltrips_v3$member_casual, FUN = mean)
aggregate(alltrips_v3$ride_length ~ alltrips_v3$member_casual, FUN = median)
aggregate(alltrips_v3$ride_length ~ alltrips_v3$member_casual, FUN = max)
aggregate(alltrips_v3$ride_length ~ alltrips_v3$member_casual, FUN = min)

Calculate the mean trip length by rider type for each day of the week.


In [ ]:
aggregate(alltrips_v3$ride_length ~ alltrips_v3$member_casual + alltrips_v3$day_of_week, FUN = mean)

Sort days of week


In [ ]:
alltrips_v3$day_of_week <- ordered(alltrips_v3$day_of_week, levels=c("Sunday", "Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday"))

Determine average ride time by each day by type of rider sorted by day


In [ ]:
mean(alltrips_v3$ride_length)


Count trips by casual and member riders


In [ ]:
table(alltrips_v3$member_casual)


Add a column for ride length in minutes.  An 8 minute ride is easier to envision than a 492 second ride.


In [ ]:
alltrips_v3  <- mutate(alltrips_v3, ride_length_minutes = round_half_up(ride_length/60))

Final check of number of rows/columns.


In [ ]:
dim(alltrips_v3)


## Export Data

Create an exportable csv file for use in Tableau.

In [ ]:
write.csv(alltrips_v3, "alltrips_final", row.names=TRUE)
